# Project 2
## Trevis Slagowski & Treyson Grange

In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt

In [ ]:
zipcodes_df = pd.read_csv('datasets/AustinZipCodes.csv')
crime_df = pd.read_csv('datasets/crime-housing-austin-2015.csv')

## Data Exploration

In [ ]:
display(zipcodes_df.head())

In [ ]:
display(crime_df.head())

In [ ]:
crime_df.columns

In [ ]:
crime_df.describe()

In [ ]:
crime_df.info()

In [ ]:
display(crime_df['Location'].value_counts())

In [ ]:
display(crime_df['Clearance_Status'].value_counts())

## First Analysis

In [ ]:
crime_df['households'] = crime_df['Largehouseholds(5+members)'].str.replace('%', '').astype('float')
display(crime_df['households'])

## Second Analysis Bulgary success rate compared to Median Household Income
(Burglaries are defined as "BURGLARY OF RESIDENCE" by the Highest_Offense_Desc column)

In [ ]:
import seaborn as sns
# Does the houshold income correlate to how much bulguaries happen in the area?
crime_df['householdincome'] = crime_df['Medianhouseholdincome'].str.replace('$', '').astype('float')
houseHoldIncome = crime_df['householdincome']
print(houseHoldIncome.shape)


pre = crime_df[crime_df['Highest_NIBRS_UCR_Offense_Description'] == 'Burglary']
burglarie = pre[crime_df['Highest_Offense_Desc'] == 'BURGLARY OF RESIDENCE']



plt.figure(figsize=(10, 6))
plt.hist(burglarie['householdincome'], bins=20, color='blue', edgecolor='black')
plt.xlabel('Household Income')
plt.ylabel('Number of Burglaries')
plt.title('Distribution of Burglaries by Household Income')
plt.show()


According to our histogram, we see almost all burglaries happen in the $40000 - $60000. Makes sense that's where most people live. Lets normalize and focus on cases that have been cleared. 

In [ ]:
income_bins = [0, 30000, 60000, 90000, 120000, 150000]
income_labels = ['<30k', '30k-60k', '60k-90k', '90k-120k', '120k-150k']

burglarie['IncomeGroup'] = pd.cut(burglarie['householdincome'], bins=income_bins, labels=income_labels)

cleared_by_arrest = burglarie[burglarie['Clearance_Status'] == 'C']

print(f"The number of rows in the cleared_by_arrest DataFrame is: {cleared_by_arrest.shape[0]}")
print(f"The number of crimes that were not solved (O or N) is: {burglarie.shape[0] - cleared_by_arrest.shape[0]}")

total_cases = burglarie.groupby('IncomeGroup').size().reset_index(name='TotalCases')

cleared_counts = cleared_by_arrest.groupby('IncomeGroup').size().reset_index(name='ClearedCases')

merged_df = pd.merge(total_cases, cleared_counts, on='IncomeGroup', how='left')

merged_df['PercentageCleared'] = (merged_df['ClearedCases'] / merged_df['TotalCases']) * 100


sns.barplot(x='IncomeGroup', y='PercentageCleared', data=merged_df)
plt.title('Percentage of Burglaries Cleared by Arrest in Different Income Groups')
plt.xlabel('Income Group')
plt.ylabel('Percentage of Cleared Cases')
plt.show()

## So even after normalizing our data, we see that the 30k-60k have the most cleared by arrest burglaries

### But it is important to note, that these are only in the range of 4-9% clear rate.
That means that very little burglaries get solved in the Austin Texas area. 
Whether this be because of police department negligence or lack of security in areas.

### Why is this important?
Even after normalizing our data, we see that the 30k - 60k has the best chance of ending in an arrest.
- For the <30k group, it is most likely because they either have poor security making the break in easier, or just don't care enough when the police ask for more information
- For the other end, 120K - 150K, it is less obvious. A thought could be that rich people seem to never be home, causing an influx of unsolved cases.

## Third Analysis

### Rental units affordable to an average teacher and the frequency of crime types in those Council Districts

In [ ]:
display(crime_df['Highest_NIBRS_UCR_Offense_Description'].unique())

In [ ]:
display(crime_df['Council_District'].value_counts())

#### First, we need to process the teacher rental housing affordability data. We can take the mean of the affordability percentage in each Council District.

In [ ]:
def percentage_to_float(percentage_str):
    if isinstance(percentage_str, str):
        return float(percentage_str.strip('%')) / 100
    return np.nan

crime_df['Rental_Affordable_to_Teacher'] = crime_df['Rentalunitsaffordabletoaverageteacher'].apply(percentage_to_float)

average_affordability = crime_df.groupby('Council_District')['Rental_Affordable_to_Teacher'].mean()
average_affordability.plot(kind='bar', ylabel='Rental Units affordable to Average Teacher', title='Average Affordability by Council District')   

#### Next, we need to aggregate crime data. We will count the number of burglaries in each Council District.

In [ ]:
unique_crimes = crime_df['Highest_NIBRS_UCR_Offense_Description'].unique()

crime_data_dict = {}

for crime in unique_crimes:
    crime_counts = crime_df[crime_df['Highest_NIBRS_UCR_Offense_Description'] == crime].groupby('Council_District').size()
    merged_data = pd.DataFrame({
        f'{crime}_Count': crime_counts,
        'Average_Affordability': average_affordability
    }).dropna()
    crime_data_dict[crime] = merged_data

unique_crimes

#### Let's plot the results in a scatter plot

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_crimes)))

for i, crime in enumerate(unique_crimes):
    ax.scatter(x=crime_data_dict[crime]['Average_Affordability'], y=crime_data_dict[crime][f'{crime}_Count'],
               color=colors[i], label=crime)

ax.set_xlabel('Rental Affordability to Average Teachers')
ax.set_ylabel('Crime Count')
ax.set_title('Crime Count vs. Housing Affordability')
ax.legend(title='Crime Types', bbox_to_anchor=(1, 1), loc='upper left')

plt.show()

#### Check the for any correlations between Rental Affordability to Average Teachers and the Crime Type

In [ ]:
correlation_dict = {}

for crime in unique_crimes:
    crime_count_col = f'{crime}_Count'

    correlation_dict[crime] = stats.pearsonr(crime_data_dict[crime][crime_count_col], crime_data_dict[crime]['Average_Affordability'])

for correlation in correlation_dict: 
    print(f"{correlation}: {correlation_dict[correlation]}")

It looks like there is a correlation for Burglary, Agg Assault, and Murder. Let's look more closely at those. 

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=crime_data_dict['Burglary']['Average_Affordability'], y=crime_data_dict['Burglary']['Burglary_Count'], data=crime_data_dict['Burglary'])
plt.title('Scatterplot of Burglary Count vs. Housing Affordability')
plt.xlabel('Rental Affordability to Average Teachers')
plt.ylabel('Burglary Count')
plt.show()

print(f"Burglary: {correlation_dict['Burglary']}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=crime_data_dict['Agg Assault']['Average_Affordability'], y=crime_data_dict['Agg Assault']['Agg Assault_Count'], data=crime_data_dict['Agg Assault'])
plt.title('Scatterplot of Agg Assault Count vs. Housing Affordability')
plt.xlabel('Rental Affordability to Average Teachers')
plt.ylabel('Agg Assault Count')
plt.show()

print(f"Agg Assault: {correlation_dict['Agg Assault']}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=crime_data_dict['Murder']['Average_Affordability'], y=crime_data_dict['Murder']['Murder_Count'], data=crime_data_dict['Murder'])
plt.title('Scatterplot of Murder Count vs. Housing Affordability')
plt.xlabel('Rental Affordability to Average Teachers')
plt.ylabel('Murder Count')
plt.show()

print(f"Murder: {correlation_dict['Murder']}")

There appears to be a statistically significant relationship between the crime rates of burglary, aggravated assault, and murder, and the rental affordability to an average teacher in different council districts. A positive coefficient indicates that as the rental affordability increases, so does the crime rate. But we can not definitively say this is the only attribute contributing to crime rates in Council Districts. 

## Fourth Analysis
### Crime Statistics by Location Data